# Credential Schema and Definitions

A credential schema has a name and version and defines a set of attribute names that once written to the ledger can be used to issue credentials following that schema. First though, all agents wishing to issue credentials following a particular schema must write a credential definition (also called claim definition) to the ledger. The credential definition is the public key for a specific agent for a specific credential schema, currently Hyperledger stack uses [CL Signatures](https://groups.csail.mit.edu/cis/pubs/lysyanskaya/cl02b.pdf). By using this type of signature scheme we are able to sign each attribute in the credential individually, making selective disclosure and efficient zero knowledge proof generation from attributes possible.

Here is an example written on the Sovrin MainNet:
* [Credential Schema](https://indyscan.io/tx/SOVRIN_MAINNET/domain/54679)
* [Credential Definition](https://indyscan.io/tx/SOVRIN_MAINNET/domain/54680)

**It is important to keep in mind that only Public DIDs with write access to the ledger are able to write transactions such as schema or definitions to the ledger. The Sovrin MainNet for example is a public permissioned network.**

In this notebook we will see how to use the AriesAgentController to write credential schema and definitions to the ledger and learn some important restrictions to be aware of.

## 1. Instantiate the Controller

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)


IPython autoawait is `on`, and set to use `asyncio`


## 2. Define schema arguments

These can be anything you want. Be Creative :)

In [2]:
# Define you schema name - must be unique on the ledger
schema_name = "alice_test_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]


## 3. Write the Schema to Ledger

A schema with the same name can only be written to the ledger once. You will not be able to run this block twice.

In [3]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:alice_test_schema:0.0.1


## 4. Write a credential definition for this schema

It is only possible to write a single credential definition per schema per public DID to the ledger. However, [Bob](http://localhost:8889/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credentials.ipynb) is also able to write a credential definition for the schema if he wants. Feel free to copy the schema_id across and give it a try in a new notebook.

#### Arguments
* schema_id - you need to pass in one of these and it must exist on the ledger
* tag - tag to identity the definition later, defaults to default
* support_revocation - Determines whether you will be able to later revoke the credential, defaults to False

**Note: You should be able to see these transactions on the local network [here](http://localhost:9000/browse/domain)**

In [ ]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

## 5. Update the Schema

You can update a schema on the ledger by changing the version.

In [ ]:
attributes = ["name", "dob", "skill"]
schema_version = "0.0.2"
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
updated_schema_id = response["schema_id"]
print(updated_schema_id)

## 6. Write another Credential Definition for the Updated Schema

Alice will now be able to issue credentials using both versions of the credential schema. Although, as a verifier you probably will expect all credentials presented to be of the latest version schema written to the ledger.

In [ ]:
response = await agent_controller.definitions.write_cred_def(updated_schema_id, support_revocation=True)

updated_cred_def_id = response["credential_definition_id"]
print(updated_cred_def_id)

## Additional Methods

Typically all you will care about is using the schema and definitions controller to write them to the ledger. The api also includes some additional helper methods that you might include in more complex flows.

## Get Schema By ID

In [ ]:
schema = await agent_controller.schema.get_by_id(schema_id)
print(schema)

## Get Created definition by id

As you can see it contains a bunch of BIG numbers all defined in the signature scheme. Notice that 'r' contains numbers for name, skill and master_secret, all attributes within the credential schema. (All credential schemas contain a master_secret attribute to identify the holder of the credential, it is blindly signed by the issuer). 

In [ ]:
definition = await agent_controller.definitions.get_by_id(cred_def_id)
print(definition)

## Search ledger for schema

This might be used by applications that want to ensure they are using the latest version of a particular schema that they know the name of. 

### Returns a list of schema ids

#### Arguments:
* schema_id - if searching for a particular set of scheme (many versions)
* schema_issuer_did - returns a set of schema created by a particular DID. Defaults to the agents DID
* schema_name - name of schema you are looking for
* schema_version - particular version of a schema

In [ ]:
created_schema = await agent_controller.schema.get_created_schema()
print(created_schema)

In [ ]:
created_schema = await agent_controller.schema.get_created_schema(schema_id=schema_id)
print(created_schema)

In [ ]:
created_schema = await agent_controller.schema.get_created_schema(schema_version='0.0.1')
print(created_schema)

In [ ]:

created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)

## Search all definitions created on the ledger

#### Optional Arguments
* schema_id
* schema_issuer_did
* schema_version
* schema_name
* issuer_did
* cred_def_id

In [ ]:
definitions = await agent_controller.definitions.search_created(cred_def_id=cred_def_id)
print(definitions)

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)